## System Setup

In [ ]:
# Find CPU info
!grep -m1 'model name' /proc/cpuinfo | awk -F": " '{print $2}'
!grep 'cpu cores' /proc/cpuinfo | awk -F": " '{a[cores]+=$2}END{print "CPU cores: " a[cores]}'

Intel(R) Xeon(R) CPU @ 2.30GHz
CPU cores: 8


In [ ]:
# Find Ram Info
!grep MemTotal /proc/meminfo | awk '{printf "%.1fGB RAM", $2 / 1024 / 1024}'

25.5GB RAM

In [ ]:
# Find GPU info
!nvidia-smi

Fri Jul 21 05:34:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python --version

Python 3.10.6


In [ ]:
!R --version

R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under the terms of the
GNU General Public License versions 2 or 3.
For more information about these matters see
https://www.gnu.org/licenses/.



In [ ]:
#Install torch manually from prebuilt binaries since colab use cu11.8 which does not build
!wget https://storage.googleapis.com/torch-lantern-builds/packages/cu117/0.11.0/src/contrib/torch_0.11.0_R_x86_64-pc-linux-gnu.tar.gz
!Rscript -e 'install.packages("torch_0.11.0_R_x86_64-pc-linux-gnu.tar.gz", repos = NULL)'

--2023-07-21 05:34:16--  https://storage.googleapis.com/torch-lantern-builds/packages/cu117/0.11.0/src/contrib/torch_0.11.0_R_x86_64-pc-linux-gnu.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 173.194.198.128, 173.194.74.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/gzip]
Saving to: ‘torch_0.11.0_R_x86_64-pc-linux-gnu.tar.gz’

torch_0.11.0_R_x86_     [    <=>             ]   1.83G   151MB/s    in 13s     

2023-07-21 05:34:29 (143 MB/s) - ‘torch_0.11.0_R_x86_64-pc-linux-gnu.tar.gz’ saved [1966658359]

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
* installing *binary* package ‘torch’ ...
* DONE (torch)


In [ ]:
# Install BKTR
!Rscript -e "library(devtools); devtools::install_github('julien-hec/BKTR', ref = 'main')"

Loading required package: usethis
cpp11       (0.4.4  -> 0.4.5  ) [CRAN]
openssl     (2.0.6  -> 2.1.0  ) [CRAN]
curl        (5.0.0  -> 5.0.1  ) [CRAN]
Rcpp        (NA     -> 1.0.11 ) [CRAN]
sp          (NA     -> 2.0-0  ) [CRAN]
png         (NA     -> 0.1-8  ) [CRAN]
isoband     (0.2.6  -> 0.2.7  ) [CRAN]
bit         (4.0.4  -> 4.0.5  ) [CRAN]
bitops      (NA     -> 1.0-7  ) [CRAN]
digest      (0.6.32 -> 0.6.33 ) [CRAN]
jpeg        (NA     -> 0.1-10 ) [CRAN]
plyr        (NA     -> 1.8.8  ) [CRAN]
RgoogleMaps (NA     -> 1.4.5.3) [CRAN]
collections (NA     -> 0.3.7  ) [CRAN]
coro        (NA     -> 1.0.3  ) [CRAN]
ggmap       (NA     -> 3.0.2  ) [CRAN]
R6P         (NA     -> 0.3.0  ) [CRAN]
Installing 17 packages: cpp11, openssl, curl, Rcpp, sp, png, isoband, bit, bitops, digest, jpeg, plyr, RgoogleMaps, collections, coro, ggmap, R6P
Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/cpp11_0.4.5.tar.gz'
Conte

## Imputation Simu Analysis (R)

In [ ]:
sh = """
library(BKTR)
library(data.table)
df_res_arr <- c()
res_colnames <- c('Lengthscale', 'Missing', 'Iter', 'B_MAE', 'B_RMSE', 'Y_MAE', 'Y_RMSE', 'Time')

RANK_DECOMP = 10
BURN_IN_ITER = 500
SAMPLING_ITER = 500

TSR$set_params('float32', 'cuda', 1)

for (len_scale in c(3, 6)) {
  for (miss_perc in c(0.1, 0.5, 0.9)) {
    for (i in 1:10) {
      spatial_kernel <- KernelMatern$new(
        smoothness_factor = 5,
        lengthscale = KernelParameter$new(value = len_scale)
      )
      temporal_kernel <- (
        KernelSE$new(lengthscale = KernelParameter$new(value = len_scale))
      )

      simu_data <- simulate_spatiotemporal_data(
        100, 150, 2, 10, 10,
        c(0, 2, 4), c(1, 3),
        spatial_kernel, temporal_kernel, 1
      )

      data_df <- simu_data$data_df
      index_choices_tsr <- TSR$tensor(1:nrow(data_df))
      nb_miss_index <- round(miss_perc * nrow(data_df))
      na_index <- as.numeric(TSR$rand_choice(index_choices_tsr, nb_miss_index)$cpu())
      data_df$y[na_index] <- NA

      bktr_regressor <- BKTRRegressor$new(
        data_df = data_df,
        rank_decomp = RANK_DECOMP,
        burn_in_iter = BURN_IN_ITER,
        sampling_iter = SAMPLING_ITER,
        spatial_kernel = KernelMatern$new(smoothness_factor = 5),
        spatial_positions_df = simu_data$spatial_positions_df,
        temporal_kernel = KernelSE$new(),
        temporal_positions_df = simu_data$temporal_positions_df,
        has_geo_coords = FALSE
      )

      bktr_regressor$mcmc_sampling()

      # Calc Beta Errors
      y_err <- bktr_regressor$imputed_y_estimates$y[na_index] - simu_data$data_df$y[na_index]
      beta_err <- unlist(abs(
        lapply(bktr_regressor$beta_estimates[, -c(1, 2)], as.numeric)
        - simu_data$beta_df[, -c(1, 2)]
      ))
      y_rmse <- sqrt(mean(y_err^2))
      y_mae <- mean(abs(y_err))
      beta_rmse <- sqrt(mean(beta_err^2))
      beta_mae <- mean(abs(beta_err))

      # Formatting Values
      df_res_arr <- c(
        df_res_arr,
        len_scale,
        miss_perc,
        sprintf('%04d', i),
        sprintf('%.4f', beta_mae),
        sprintf('%.4f', beta_rmse),
        sprintf('%.4f', y_mae),
        sprintf('%.4f', y_rmse),
        sprintf('%.3f', as.numeric(bktr_regressor$result_logger$total_elapsed_time,units="secs"))
      )
      df <- as.data.table(matrix(df_res_arr, ncol = length(res_colnames), byrow = TRUE))
      colnames(df) <- res_colnames
      print(df)
    }
  }
}
"""
with open('5_4_simu_imputation.R', 'w') as file:
  file.write(sh)
!Rscript 5_4_simu_imputation.R

Streaming output truncated to the last 5000 lines.
[1] "Iter 308   | Elapsed Time:     0.19s | MAE:  0.7862 | RMSE:  0.9869"
[1] "Iter 309   | Elapsed Time:     0.15s | MAE:  0.7964 | RMSE:  0.9981"
[1] "Iter 310   | Elapsed Time:     0.18s | MAE:  0.8018 | RMSE:  1.0067"
[1] "Iter 311   | Elapsed Time:     0.24s | MAE:  0.7836 | RMSE:  0.9935"
[1] "Iter 312   | Elapsed Time:     0.21s | MAE:  0.7846 | RMSE:  0.9912"
[1] "Iter 313   | Elapsed Time:     0.15s | MAE:  0.7856 | RMSE:  0.9827"
[1] "Iter 314   | Elapsed Time:     0.15s | MAE:  0.7840 | RMSE:  0.9874"
[1] "Iter 315   | Elapsed Time:     0.13s | MAE:  0.7848 | RMSE:  0.9855"
[1] "Iter 316   | Elapsed Time:     0.31s | MAE:  0.7933 | RMSE:  0.9970"
[1] "Iter 317   | Elapsed Time:     0.12s | MAE:  0.7970 | RMSE:  1.0079"
[1] "Iter 318   | Elapsed Time:     0.13s | MAE:  0.7916 | RMSE:  0.9950"
[1] "Iter 319   | Elapsed Time:     0.24s | MAE:  0.7873 | RMSE:  0.9955"
[1] "Iter 320   | Elapsed Time:     0.14s | MAE:  0.7853 | RM